In [22]:
# Configuração inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')

print("Ambiente configurado com sucesso!")

Ambiente configurado com sucesso!


In [23]:
# Análise de Gastos Municipais
## Dados do Tesouro Nacional

print("**Autor**: Roberto da Cunha")
print("**Data**: 2025-08-03")  # Números em strings não causam erro

**Autor**: Roberto da Cunha
**Data**: 2025-08-03


In [24]:
# Adicionar no final da função, antes de salvar
def validate_data(df):
    """Validações básicas da qualidade dos dados"""
    assert not df.empty, "DataFrame vazio após processamento"
    assert df["ano_exercicio"].between(2010, 2023).all(), "Anos fora do intervalo esperado"
    monetary_cols = ["valor_atualizado", "valor_orcado", "valor_liquidado"]
    for col in monetary_cols:
        if col in df.columns:
            assert df[col].ge(0).all(), f"Valores negativos encontrados em {col}"
    return True

In [25]:
# Adicionar no início da função
PARAMS = {
    "encoding": "ISO-8859-1",
    "sep": ";",
    "decimal": ",",
    "na_values": ["-", "NA", ""],
    "min_year": 2010,
    "max_year": 2023
}

def generate_documentation():
    """Gera documentação automática do processamento"""
    doc = f"""
    # Documentação do Pré-processamento
    
    Parâmetros utilizados:
    - Encoding: {PARAMS['encoding']}
    - Separador: {PARAMS['sep']}
    - Decimal: {PARAMS['decimal']}
    - Valores considerados como NA: {PARAMS['na_values']}
    - Intervalo de anos válidos: {PARAMS['min_year']}-{PARAMS['max_year']}
    """
    with open("../docs/preprocessing_report.md", "w") as f:
        f.write(doc)

In [28]:
# Modificação para arquivos muito grandes
def process_large_file(raw_path, chunksize=10000):
    """Processa arquivos grandes em chunks"""
    chunks = []
    for chunk in pd.read_csv(raw_path, chunksize=chunksize, encoding=PARAMS["encoding"], sep=PARAMS["sep"]):
        # Aplicar todas as transformações aqui
        chunks.append(processed_chunk)
    return pd.concat(chunks, ignore_index=True)

In [29]:
def process_large_file(raw_path, chunksize=10000):
    """Processa arquivos grandes em chunks com todas as transformações"""
    chunks = []
    
    for chunk in pd.read_csv(
        raw_path,
        chunksize=chunksize,
        encoding=PARAMS["encoding"],
        sep=PARAMS["sep"],
        decimal=PARAMS["decimal"],
        dtype={"codigo_ibge": str},
        thousands=".",
        na_values=PARAMS["na_values"]
    ):
        # Aplica todas as transformações em cada chunk
        processed_chunk = clean_column_names(chunk)
        processed_chunk = convert_monetary_values(processed_chunk)
        
        # Filtro de anos (se aplicável)
        if "ano_exercicio" in processed_chunk.columns:
            processed_chunk = processed_chunk[
                processed_chunk["ano_exercicio"].between(PARAMS["min_year"], PARAMS["max_year"])
            ]
        
        chunks.append(processed_chunk)
    
    # Combina todos os chunks processados
    return pd.concat(chunks, ignore_index=True)